## Part 1: Preprocessing

In [87]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [88]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [89]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

y_df

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [90]:
# Create a list of at least 10 column names to use as X data
column_list = ['Education',
               'Age',
               'DistanceFromHome',
               'JobSatisfaction',
               'OverTime',
               'StockOptionLevel',
               'WorkLifeBalance',
               'YearsAtCompany',
               'YearsSinceLastPromotion',
               'NumCompaniesWorked'
               ]

# Create X_df using your selected columns
X_df = attrition_df[column_list]


# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [91]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=29)

In [92]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

OverTime
No     791
Yes    311
Name: count, dtype: int64

In [93]:
X_train['OverTime'] = X_train['OverTime'].map({'No': 0, 'Yes': 1})
X_test['OverTime'] = X_test['OverTime'].map({'No': 0, 'Yes': 1})

In [94]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [95]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_ohe = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_department_ohe = department_ohe.fit(y_train['Department'].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dep_encoded = y_department_ohe.transform(y_train['Department'].values.reshape(-1,1))
y_test_dep_encoded = y_department_ohe.transform(y_test['Department'].values.reshape(-1,1))

y_train_dep_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [96]:
# Create a OneHotEncoder for the Attrition column
attrition_ohe = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_attrition_ohe = attrition_ohe.fit(y_train['Attrition'].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr_encoded = y_attrition_ohe.transform(y_train['Attrition'].values.reshape(-1,1))
y_test_attr_encoded = y_attrition_ohe.transform(y_test['Attrition'].values.reshape(-1,1))

y_train_attr_encoded

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

## Part 2: Create, Compile, and Train the Model

In [97]:
# Find the number of columns in the X training data.
num_columns = len(X_train.columns)

# Create the input layer
input_layer = tf.keras.layers.Input(shape=(num_columns,), name='input')

# Create at least two shared layers
shared_layer1 = tf.keras.layers.Dense(units=64, activation='relu', name='shared_1')(input_layer)
shared_layer2 = tf.keras.layers.Dense(units=128, activation='relu', name='shared_2')(shared_layer1)

In [98]:
y_train['Department'].value_counts()

Department
Research & Development    727
Sales                     330
Human Resources            45
Name: count, dtype: int64

In [99]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_layer = tf.keras.layers.Dense(units=32, activation='tanh', name='department_hidden')(shared_layer2)

# Create the output layer
department_output_layer = tf.keras.layers.Dense(units=3, activation='softmax', name='department_output')(department_layer)

In [100]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_layer = tf.keras.layers.Dense(units=32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output_layer = tf.keras.layers.Dense(units=2, activation='sigmoid', name='attrition_output')(attrition_layer)

In [101]:
# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
                    metrics={'department_output': 'accuracy',
                             'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 10)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_1 (Dense)    │ (None, 64)        │        704 │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_2 (Dense)    │ (None, 128)       │      8,320 │ shared_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared_2[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared_2[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [102]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_dep_encoded,
     'attrition_output': y_train_attr_encoded},
     epochs=10
)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - attrition_output_accuracy: 0.6122 - attrition_output_loss: 0.6452 - department_output_accuracy: 0.5936 - department_output_loss: 0.8916 - loss: 1.5371   
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8299 - attrition_output_loss: 0.4490 - department_output_accuracy: 0.6709 - department_output_loss: 0.7298 - loss: 1.1786 
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8499 - attrition_output_loss: 0.3908 - department_output_accuracy: 0.6655 - department_output_loss: 0.7410 - loss: 1.1320 
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8456 - attrition_output_loss: 0.3781 - department_output_accuracy: 0.6486 - department_output_loss: 0.7620 - loss: 1.1400 
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8565 - attrition_output_loss: 0.3741 - department_output_accuracy: 0.6591 - department_output_loss: 0

In [103]:
# Evaluate the model with the testing data
eval_metrics = model.evaluate(X_test_scaled,
                                {'department_output': y_test_dep_encoded,
                                 'attrition_output': y_test_attr_encoded},
                                 verbose=2)
print(f'[{eval_metrics[0]}, {eval_metrics[3]}, {eval_metrics[4]:.1}]')

12/12 - 0s - 13ms/step - attrition_output_accuracy: 0.8696 - attrition_output_loss: 0.3720 - department_output_accuracy: 0.5679 - department_output_loss: 0.8414 - loss: 1.2226
[1.2226431369781494, 0.8695651888847351, 0.6]


In [104]:
# Print the accuracy for both department and attrition
print(f'Attrition predictions accuracy: {eval_metrics[3]}')
print(f'Department predictions accuracy: {eval_metrics[4]:.1}')

Attrition predictions accuracy: 0.8695651888847351
Department predictions accuracy: 0.6


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Is accuracy the best metric to use on this data? Why or why not?

* Accuracy might not be the best metric for both tasks, especially if there is a significant class imbalance in either the attrition predictions (far fewer employees leave than stay) or the department predictions (some departments might be much larger than others).

    * Attrition Predictions: While an accuracy of ~87% seems good, if the dataset is imbalanced, a model could achieve high accuracy by simply predicting the majority class most of the time. Metrics like precision, recall, F1-score, and the Area Under the ROC Curve (AUC) would provide a more comprehensive view of the model's performance, especially in identifying employees likely to leave.
    * Department Predictions: An accuracy of 0.6 (60%) might be acceptable depending on the number of departments. However, if some departments are much more frequent than others, the model might be biased towards predicting those. Again, metrics like precision, recall, and F1-score (potentially macro or weighted averages) for each department would give a better understanding of the model's ability to correctly classify employees into different departments. A confusion matrix would also be very helpful to see which departments are being misclassified.
#
2. What activation functions did you choose for your output layers, and why?

    * For the Attrition predictions output layer, the activation function I used was 'sigmoid'. This is because attrition is a binary classification problem, and the sigmoid function outputs a probability between 0 and 1.
    * For the Department predictions output layer, the activation function I used is 'softmax'. This is because department prediction is a multi-class classification problem, and softmax provides a probability distribution over all possible departments.
#
3. Can you name a few ways that this model might be improved?

* Here are several ways the model could potentially be improved:


    * Hyperparameter Tuning: Experiment with different numbers of units in the shared and branch-specific hidden layers. Also, try different activation functions (e.g., 'relu' is common, but others like 'leaky_relu' or 'elu' could be explored).
    * Address Class Imbalance: If there's a significant imbalance in the attrition or department classes in your training data, consider techniques like oversampling the minority class, undersampling the majority class, or using class weights in the loss function.
    * Regularization: Add regularization techniques like L1 or L2 regularization to the dense layers or use dropout layers to prevent overfitting.
    * Increase Data: More data, if available, can often lead to better model performance.
    * Feature Engineering: Create new features from the existing data that might be more predictive of attrition or department.
    * Different Model Architectures: Explore slightly different network structures, perhaps adding more shared layers or adjusting the number of layers in the branches.
    * Learning Rate Tuning: Experiment with different learning rates for the optimizer (Adam).
    * Evaluate with More Metrics: As mentioned in question 1, use a wider range of metrics beyond just accuracy, such as precision, recall, F1-score, and AUC (for attrition), and per-class precision, recall, and F1-score, and a confusion matrix (for department).